Lets try extracting all the professors name, office number, research interests and education details from snu website

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import getpass

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

model1 = init_chat_model("openai/gpt-oss-20b", model_provider="groq")

In [5]:
from langchain_unstructured import UnstructuredLoader

loader=UnstructuredLoader(web_url="https://seas.harvard.edu/computer-science/courses") #pass the webpage we want the information to be extracted from
docs = loader.load()

In [6]:
prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. Present the data, if any, in tabular form with a heading in bold.
Question: {question} 
Context: {context} 
Answer:"""

In [7]:
full_doc = "\n\n".join(doc.page_content for doc in docs)
print(full_doc)

Computer Science

Course Listing

For a snapshot of courses being offered by Harvard School of Engineering over the next four years, visit our Multi Year Course Planning tool.

Great Ideas in Computer Science

COMPSCI 1 2026 Spring

Henry Leitner Tuesday, Thursday 10:30am to 11:45am

An introduction to the most important discoveries and intellectual paradigms in computer science, designed for students with little or no previous background. Explores problem-solving and data analysis using Python, a programming language with a simple syntax and a powerful set of libraries. This course covers basic data types and collections (lists, dictionaries, tuples, and sets), control flow, recursion, supervised machine learning via regression, visualization, information hiding and encapsulation using classes and objects, and introduces the analysis of program performance. Presents an integrated view of computer systems, from switching circuits up through compilers, and examines theoretical and pract

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=7000,   # adjust to safe limit for your model
    chunk_overlap=200, # overlap for context continuity
    length_function=len
)

chunks = splitter.split_text(full_doc)
print(f"Total Chunks: {len(chunks)}")


Total Chunks: 13


In [9]:
question='''
Your task is to extract and return structured course information in a Markdown table.  
Each course should include the following columns:  

| Course Title | Course Code | Credits | Professor | Timings | Description | Duration |

If some fields are missing, leave them empty.  
Do not add extra commentary — only output the table.'''

import time

from langchain.chat_models import init_chat_model
llm = init_chat_model("moonshotai/kimi-k2-instruct", model_provider="groq")

result=[]
for chunk in chunks:
    response = llm.invoke(prompt_template.format(
    context=chunk,
    question=question))
    result.append(response.content)

    time.sleep(10) #delay of 2 s between each chiunk is processed
    
final_output = result[0].strip()
for res in result[1:]:
    # Skip header rows from subsequent chunks
    final_output += "\n" + "\n".join(res.strip().split("\n")[2:])

from IPython.display import Markdown
Markdown(final_output)

INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

| Course Title | Course Code | Credits | Professor | Timings | Description | Duration |
|--------------|-------------|---------|-----------|---------|-------------|----------|
| Great Ideas in Computer Science | COMPSCI 1 |  | Henry Leitner | Tuesday, Thursday 10:30am–11:45am | Intro to key CS discoveries & Python problem-solving, from data types to machine learning & ethics. | 2026 Spring |
| Discrete Mathematics for Computer Science | COMPSCI 20 |  | Rebecca Nesson | Monday, Wednesday, Friday 9:45am–11:00am | Logic, set theory, combinatorics, number theory, probability, graph theory; formal proofs. | 2026 Spring |
| Computational Thinking and Problem Solving | COMPSCI 32 |  | Kitty Ascrizzi | Monday, Wednesday 1:30pm–2:45pm | Python-based intro to computational thinking & real-world problem solving, no prior experience needed. | 2026 Spring |
| Introduction to Computer Science | COMPSCI 50 |  | David J. Malan | Monday 1:30pm–4:15pm | CS50: broad intro to CS concepts, C → Python → web tech; final project. | 2025 Fall |
| Introduction to Computer Science | COMPSCI 50 |  | David J. Malan, Kelly Ding | Tuesday 3:45pm–6:30pm | CS50: broad intro to CS concepts, C → Python → web tech; final project. | 2026 Spring |
| Introduction to Computer Science | COMPSCI 50 |  | David J. Malan, Kelly Ding | Wednesday 9:00am–11:45am | CS50: broad intro to CS concepts, C → Python → web tech; final project. | 2026 Spring |
| Course Title | Course Code | Credits | Professor | Timings | Description | Duration |
|--------------|-------------|---------|-----------|---------|-------------|----------|
| Introduction to Computer Science | COMPSCI 50 |  | David J. Malan, Kelly Ding | Wed 9:00–11:45 am, Tue 9:00–11:45 am, Wed 6:00–8:45 pm | Harvard’s introduction to CS and programming; covers C, Python, SQL, HTML/CSS/JS; ends with final project. |  |
| Abstraction and Design in Computation | COMPSCI 51 |  | Stuart Shieber | Tue/Thu 12:45–2:00 pm, Tue/Thu 3:45–5:00 pm | Emphasizes abstraction across functional, imperative, and object-oriented paradigms; focuses on design principles and software engineering. |  |
| Abstraction and Design in Computation | COMPSCI 51 |  | Stuart Shieber | Tuesday, Thursday 3:45pm–5:00pm | Fundamental concepts in program design, abstraction, multi-paradigm programming, software engineering, models of computation. | 2026 Spring |
| Systems Programming and Machine Organization | COMPSCI 61 |  | Eddie Kohler | Monday, Wednesday 2:15pm–3:30pm | Systems programming, machine organization, performance tuning, C/assembly, memory hierarchy, concurrency. | 2025 Fall |
| Design of Useful and Usable Interactive Systems | COMPSCI 79 |  | Krzysztof Gajos | Monday, Wednesday 9:45am–11:00am | Human-centered design, need-finding, prototyping, team projects. | 2025 Fall |
| Supervised Reading and Research | COMPSCI 91R |  | Stephen Chong, Adam Hesterberg, Eddie Kohler |  | Individual advanced study supervised by faculty. | 2025 Fall |
| Supervised Reading and Research | COMPSCI 91R |  | Stephen Chong, Adam Hesterberg, Eddie Kohler |  | Individual advanced study supervised by faculty. | 2026 Spring |
| Privacy and Technology | COMPSCI 1050 |  | Jim Waldo | Tuesday, Thursday 12:45pm–2:00pm | Privacy concepts, technology impact, policy and ethics case studies. | 2025 Fall |
| Software Engineering with Generative AI | COMPSCI 1060 |  | Christopher Thorpe | Tuesday, Thursday 3:45pm–5:00pm | SaaS development with generative AI, CI/CD, full lifecycle. | 2025 Fall |
| Data Science 1: Introduction to Data Science | COMPSCI 1090A |  | Pavlos Protopapas, Kevin A. Rader | Monday, Wednesday 10:30am–11:45am | Data wrangling, exploratory analysis, prediction, communication; part 1 of 2. | 2025 Fall |
| Data Science 2: Advanced Topics in Data Science | COMPSCI 1090B |  | Pavlos Protopapas, Alex Young | Monday, Wednesday, Friday 9:45am–11:00am | Deep learning, Bayesian methods, unsupervised learning; part 2 of 2. | 2026 Spring |
| Introduction to Algorithms and their Limitations | COMPSCI 1200 |  | Salil Vadhan | Tuesday, Thursday 9:45am–11:00am |  | 2025 Fall |
| Introduction to Algorithms and their Limitations | COMPSCI 1200 |  | Salil Vadhan | Tuesday, Thursday 9:45am–11:00am | Introductory theoretical CS: mathematical abstraction, rigorous proof, algorithm design, correctness, efficiency, limits of computation. |  |
| Introduction to Algorithms and their Limitations | COMPSCI 1200 |  | Anurag Anshu | Monday, Wednesday 12:45pm–2:00pm | Introductory theoretical CS: mathematical abstraction, rigorous proof, algorithm design, correctness, efficiency, limits of computation. |  |
| Introduction to Theoretical Computer Science | COMPSCI 1210 |  | Adam Hesterberg | Tuesday, Thursday 3:45pm–5:00pm | Fundamental capabilities & limits of computation, universality, uncomputability, undecidability, crypto, randomness, quantum speed-ups. |  |
| Data Structures and Algorithms | COMPSCI 1240 |  | Madhu Sudan, Adam Hesterberg | Monday, Wednesday 2:15pm–3:30pm | Design & analysis of efficient algorithms and data structures: graph, approximation, randomized algorithms. |  |
| Privacy, Fairness, and Validity Through the Lens of Theoretical CS | COMPSCI 1261 |  | Cynthia Dwork | Monday, Wednesday 12:45pm–2:00pm | Theory of algorithmic fairness, differential privacy, cryptography, statistical validity. |  |
| Convex Optimization and Applications in Machine Learning | COMPSCI 1280 |  | Yiling Chen | Monday, Wednesday 11:15am–12:30pm | Convex optimization theory & algorithms with ML applications. |  |
| Economics and Computation | COMPSCI 1360 |  | Ariel Procaccia | Tuesday, Thursday 11:15am–12:30pm | Interaction of economics & CS: game theory, mechanism design, social choice, societal applications. |  |
| Computing Hardware | COMPSCI 1410 |  | Woodward Yang | Monday, Wednesday 12:45pm–2:00pm | Digital computing systems design: CMOS, logic, memory, FSMs, MIPS processor implementation. |  |
| Computer Architecture | COMPSCI 1411 |  | David Brooks | Friday 9:00am–11:45am | Modern processor structures, memory, pipelining, parallelism, GPU/datacenter/SoC/ML acceleration. |  |
| Networks Design Projects | COMPSCI 1440R |  | H. Kung | Monday, Wednesday 3:45pm–5:00pm |  |  |
| Course Title | Course Code | Credits | Professor | Timings | Description | Duration |
|--------------|-------------|---------|-----------|---------|-------------|----------|
| Networks Design Projects | COMPSCI 1440R |  | H. Kung | Monday, Wednesday 3:45pm to 5:00pm | Research-oriented course on distributed & parallel deep learning, covering communication/computation trade-offs, federated learning, low-latency inference, and full-stack optimization. 8 modules plus team project. |  |
| Design of VLSI Circuits and Systems | COMPSCI 1480 |  | Gage Hills | Monday, Wednesday 9:45am to 11:00am | Concepts and techniques for VLSI design and fabrication: MOS theory, digital MOS circuits, timing, synthesis, place & route, testing. Extensive CAD tool use. |  |
| Programming Languages | COMPSCI 1520 |  | Nada Amin | Tuesday, Thursday 11:15am to 12:30pm | Principal features of traditional and modern languages: syntax, semantics, types, modularity, polymorphism, closures, continuations, concurrency. |  |
| Compilers | COMPSCI 1530 |  | Stephen Chong | Monday, Wednesday 12:45pm to 2:00pm | Building efficient interpreters/compilers: lexical analysis, parsing, type checking, code generation, register allocation, optimization. |  |
| Operating Systems | COMPSCI 1610 |  | James Mickens | Monday, Wednesday 2:15pm to 3:30pm | Design and implementation of modern OS: threads, processes, virtual memory, schedulers, system calls, file systems, virtualization. Intensive programming assignments. |  |
| Data Systems | COMPSCI 1650 |  | Stratos Idreos | Tuesday, Thursday 9:45am to 11:00am | Comprehensive intro to modern data systems: column stores, shared-nothing architectures, in-memory systems, adaptive indexing, stream processing, key-value stores, plus traditional relational concepts. |  |
| Visualization | COMPSCI 1710 |  | Hanspeter Pfister | Monday, Wednesday 2:15pm to 3:30pm | Key principles for data visualization: design practices, perception, interaction, tools, web-based interactive programming. |  |
| Computer Graphics | COMPSCI 1750 |  | Steven Gortler | Monday, Wednesday 11:15am to 12:30pm | Fundamentals of 3D graphics with modern OpenGL: transformations, animation, rasterization, materials, textures, plus ray tracing and modeling. |  |
| Usable Interactive Systems | COMPSCI 1780 |  | Elena Glassman | Monday, Wednesday 3:45pm to 5:00pm |  |  |
| Course Title | Course Code | Credits | Professor | Timings | Description | Duration |
|--------------|-------------|---------|-----------|---------|-------------|----------|
| Usable Interactive Systems | COMPSCI 1780 |  | Elena Glassman | Monday, Wednesday 3:45pm–5:00pm | Techniques, concepts, and technologies for building usable interactive systems; covers human cognition, iterative prototyping, human-AI interaction; front- and back-end final project. |  |
| Machine Learning | COMPSCI 1810 |  | David Alvarez Melis, Yilun Du | Tuesday, Thursday 9:45am–11:00am | Probabilistic view of AI; supervised/unsupervised learning, neural networks, SVMs, graphical models; requires Python programming. |  |
| Planning and Learning Methods in AI | COMPSCI 1820 |  | Stephanie Gil, Kiante Brantley | Tuesday, Thursday 11:15am–12:30pm | Search, planning, optimization, games, uncertainty, learning; ethical AI focus; societal applications. |  |
| Introduction to Computational Linguistics and Natural-language Processing | COMPSCI 1870 |  | Stuart Shieber | Monday, Wednesday, Friday 11:15am–12:30pm | Computational linguistics tools; statistical modeling, neural networks, LLMs; build transformer-based QA system. |  |
| Designing K–12 Computer Science Learning Experiences | COMPSCI 1960 |  | Karen Brennan | Wednesday 9:00am–11:45am | K–12 CS education research and design; reading group + lab; independent project on curricula, tools, or policy. |  |
| High Performance Computing for Science and Engineering | COMPSCI 2050 |  | Chuck Witt | Tuesday, Thursday 2:15pm–3:30pm | Parallel programming models for shared/distributed memory; labs and term project. |  |
| Biology and Complexity | COMPSCI 2212 |  | Leslie Valiant | Tuesday, Thursday 12:45pm–2:00pm |  |  |
| Biology and Complexity | COMPSCI 2212 |  | Leslie Valiant | Tuesday, Thursday 12:45pm–2:00pm | Explores biological step-by-step processes using computational complexity; emphasis on evolution and neuroscience. | 2026 Spring |
| Quantum Complexity Theory | COMPSCI 2232 |  | Anurag Anshu | Tuesday, Thursday 10:30am–11:45am | Introduces quantum complexity theory focusing on quantum non-local games and many-body systems. | 2025 Fall |
| Quantum Learning Theory | COMPSCI 2233 |  | Sitan Chen, Jordan Cotler | Monday, Wednesday 3:00pm–4:15pm | Covers learning quantum systems with quantum computers; topics include quantum states, dynamics, and many-body systems. | 2025 Fall |
| Spectral Graph Theory in Computer Science | COMPSCI 2252 |  | Salil Vadhan | Monday, Wednesday 9:45am–11:00am | Studies eigenvectors/eigenvalues of graphs and applications to clustering, MCMC, web search, max-flow, etc. | 2026 Spring |
| Seminar on Pseudorandomness & High-Dimensional Expanders | COMPSCI 2253 |  | Salil Vadhan | Monday, Wednesday 12:45pm–2:00pm | Students present and critique current research papers on pseudorandomness and high-dimensional expanders. | 2026 Spring |
| Topics in Theory for Society: Differential Privacy | COMPSCI 2260 |  | Cynthia Dwork | Monday, Wednesday 9:45am–11:00am | Rigorous treatment of differential privacy: definition, primitives, composition, and applications. | 2025 Fall |
| Topics at the Interface between Computer Science and Economics | COMPSCI 2360R |  | Yiling Chen | Tuesday, Thursday 12:45pm–2:00pm | Rotating topics on computation/economics interplay: mechanism design, strategy-aware ML, social choice, etc. | 2025 Fall |
| Optimized Democracy | COMPSCI 2380 |  | Ariel Procaccia | Monday, Wednesday 11:15am–12:30pm | Mathematical and algorithmic foundations of democratic system design: voting rules, redistricting, sortition, liquid democracy. | 2025 Fall |
| Computing at Scale | COMPSCI 2420 |  | H. Kung | Monday, Wednesday 3:45pm–5:00pm | Efficient large-scale AI computation: parallel/distributed training & inference for CNNs & Transformers, model-system co-design. | 2025 Fall |
| Networks Design Projects | COMPSCI 2440R |  | H. Kung | Monday, Wednesday 3:45pm–5:00pm | Research-oriented coverage of distributed & parallel DNNs, federated learning, quantized inference, full-stack optimization, privacy/security, and emerging neuromorphic/NN-architecture-search techniques; includes substantial team project. | 2026 Spring |
| Advanced Topics in Computer Architecture | COMPSCI 2470R |  | David Brooks | Friday 9:00am–11:45am | Seminar surveying recent research in computer architecture (multi-core, energy-efficient, reliable systems); students present papers and complete a research project. | 2025 Fall |
| Advanced Design of VLSI Circuits and Systems | COMPSCI 2480 |  | Gage Hills | Monday, Wednesday 9:45am–11:00am | Concepts and CAD flows for VLSI design: MOS theory, digital circuits, timing, synthesis, P&R, testing; includes substantial design project and paper discussions. | 2026 Spring |
| Topics in Edge Computing: Architecture 2.0 – Agentic AI for Computer Systems Design | COMPSCI 2490R |  | Vijay Janapa Reddi | Monday, Wednesday 2:15pm–3:30pm | Research seminar on autonomous AI-driven compiler, processor, and chip-design methodologies; paper readings and collaborative research project. | 2025 Fall |
| Advanced Topics in Programming Languages | COMPSCI 2520R |  | Nada Amin | Tuesday, Thursday 11:15am–12:30pm | Seminar covering recent PL research; students read papers and pursue a research project. | 2025 Fall |
| Formal Methods for Computer Security | COMPSCI 2540 |  | Stephen Chong | Tuesday, Thursday 12:45pm–2:00pm | Formal models and verification for security: hyperproperties, side-channels, crypto protocols, info-flow, authorization logics; includes small and final projects. | 2026 Spring |
| Systems Security | COMPSCI 2630 |  | James Mickens | Monday, Wednesday 2:15pm–3:30pm | Practical attacks and defenses: buffer overflows, web security, Tor, anonymous communication; several small projects plus open-ended final project. | 2025 Fall |
| Big Data Systems | COMPSCI 2650 |  | Stratos Idreos | Tuesday, Thursday 9:45am–11:00am |  | 2026 Spring |
| Big Data Systems | COMPSCI 2650 |  | Stratos Idreos | Tuesday, Thursday 9:45am to 11:00am | Design of data systems, structures, and algorithms for relational, distributed, graph, NoSQL, NewSQL, ML, and neural-network domains. | 2026 Spring |
| Design, Technology, and Social Impact | COMPSCI 2760 |  | Krzysztof Gajos | Tuesday, Thursday 9:45am to 11:00am | Research and practice at the intersection of design, technology, and social impact, including intervention design and societal-impact evaluation. | 2026 Spring |
| Conceptualizing, Building, and Evaluating Usable Novel Interactive Systems | COMPSCI 2780 |  | Elena Glassman | Monday, Wednesday 3:45pm to 5:00pm | Techniques for building usable novel interactive systems, with hands-on frameworks, prototyping, and a user-evaluated research project. | 2026 Spring |
| Research Topics in Human-Computer Interaction | COMPSCI 2790R |  | Elena Glassman | Monday, Wednesday 1:30pm to 2:45pm | Reading, writing, and presenting HCI research focused on human-compatible interfaces and automation, culminating in a conference-style project. | 2025 Fall |
| Topics in Machine Learning: Compositional AI Systems | COMPSCI 2821R |  | Yilun Du | Monday, Wednesday 12:45pm to 2:00pm | Seminar surveying classical to modern compositional AI systems and their applications in robotics, NLP, reasoning, and general AI. | 2025 Fall |
| Topics in Machine Learning: Computational Properties in Interpretable Machine Learning | COMPSCI 2822R |  | Finale Doshi-Velez | Monday, Wednesday 12:45pm to 2:00pm | Computational properties (faithfulness, stability) of interpretable ML methods to guide context-specific choices without costly user studies. | 2026 Spring |
| Advanced Topics in Modern Machine Learning | COMPSCI 2823R |  | Yiling Chen, David Belanger, Alexander D'Amour, Liliya Lavitas, Roland Snoek, Zi Wang | Friday 9:45am to 12:30pm | Student-led exploration of active ML/AI research areas such as probabilistic ML, generative models, causality, and applications. | 2025 Fall |
| Computational Optimal Transport for Machine Learning | COMPSCI 2840 |  | David Alvarez Melis | Tuesday, Thursday 12:45pm to 2:00pm |  | 2025 Fall |
| Computational Optimal Transport for Machine Learning | COMPSCI 2840 |  | David Alvarez Melis | Tuesday, Thursday 12:45pm–2:00pm | Foundations & computation of Optimal Transport for ML: Monge/Kantorovich, entropy regularization, Gromov-Wasserstein, domain adaptation, generative models. |  |
| Multi-Robot Systems: Control, Communication, and Security | COMPSCI 2860 |  | Stephanie Gil | Monday, Wednesday 11:15am–12:30pm | Seminar on control, coordination & security of multi-agent robotic systems with theory and applications. |  |
| AI for Social Impact | COMPSCI 2880 |  | Milind Tambe | Monday, Wednesday 2:15pm–3:30pm | Case studies & foundations of deploying AI for public health, sustainability, safety; field projects with non-profits. |  |
| Topics in Foundations of ML: AI Alignment and Safety | COMPSCI 2881R |  | Boaz Barak | Thursday 3:45pm–6:30pm | Graduate survey of technical & societal challenges in AI alignment and safety; heavy use of AI tools expected. |  |
| Seminar on Effective Research Practices and Academic Culture | COMPSCI 2901 | 4 | Boaz Barak, John Girash | Friday 9:45am–11:45am | Year-long Ph.D. seminar on research skills, advising, academic culture; must complete CS2901+CS2902 same year. | Full-year |
| Seminar on Effective Research Practices and Academic Culture | COMPSCI 2902 | 4 |  | Friday 9:45am–11:45am | Continuation of CS2901; see CS2901 for details. | Full-year |
| Special Topics in Computer Science | COMPSCI 2990R |  | Leslie Valiant |  | Individual research or reading supervised by SEAS faculty; arrangement required before enrolment. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | Jim Waldo |  | Individual research or reading supervised by SEAS faculty; arrangement required before enrolment. |  |
| Course Title | Course Code | Credits | Professor | Timings | Description | Duration |
|--------------|-------------|---------|-----------|---------|-------------|----------|
| Special Topics in Computer Science | COMPSCI 2990R |  | Jim Waldo |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | Nada Amin |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | Kiante Brantley |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | David Brooks |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | Elena Glassman |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | Steven Gortler |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | Sham Kakade |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | Eddie Kohler |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | James Mickens |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | David Alvarez Melis |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | Ariel Procaccia |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | Stuart Shieber |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | Milind Tambe |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Special Topics in Computer Science | COMPSCI 2990R |  | Finale Doshi-Velez |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. |  |
| Course Title | Course Code | Credits | Professor | Timings | Description | Duration |
|--------------|-------------|---------|-----------|---------|-------------|----------|
| Special Topics in Computer Science | COMPSCI 2990R |  | Finale Doshi-Velez |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Fernanda Viegas |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Yilun Du |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Martin Wattenberg |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Cynthia Dwork |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Juncheng Yang |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Krzysztof Gajos |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Minlan Yu |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Stephanie Gil |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Yannai Gonczarowski |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Anurag Anshu |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Stratos Idreos |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Boaz Barak |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Hima Lakkaraju |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Sitan Chen |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Michael Mitzenmacher |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Course Title | Course Code | Credits | Professor | Timings | Description | Duration |
|--------------|-------------|---------|-----------|---------|-------------|----------|
| Special Topics in Computer Science | COMPSCI 2990R |  | Yiling Chen |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Susan Murphy |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Stephen Chong |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | David Parkes |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Hanspeter Pfister |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Michael Smith |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Madhu Sudan |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Salil Vadhan |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2025 Fall |
| Special Topics in Computer Science | COMPSCI 2990R |  | Madhu Sudan |  | Experimental or theoretical research project on acceptable problems in computer science supervised by a SEAS faculty member, and/or supervised reading on topics not covered by regular courses of instruction. The project or reading must be arranged between the student and individual SEAS faculty supervisor prior to enrolling in the course. | 2026 Spring |